In [1]:
import os
import pandas as pd 
import numpy as np 

In [113]:

df =pd.read_csv('FileList.csv')
df['DummyID'] = df.filename.apply(lambda x : int(x.split("_")[0]))

In [114]:
df.head()

,filename,DummyID
0,76254864_L_MLO_1.jpg,76254864
1,76869974_L_MLO_1.jpg,76869974
2,75980282_R_MLO_1.jpg,75980282
3,75365498_L_CC_1.jpg,75365498
4,76899031_L_CC_1.jpg,76899031


In [87]:
truthFile = 'TargetFiles/TargetFile_Combined.csv'
truthDf = pd.read_csv(truthFile)

truthDf.head()

,Age,BMI,Density_Overall,DummyID,Medview_Race,ScreenDate
0,67.8,27.1,2.0,2508626,White,3/23/2011
1,58.8,25.7,2.0,2505738,White,6/2/2011
2,56.4,28.1,2.0,2547962,White,5/24/2011
3,51.7,31.8,2.0,2575490,White,5/6/2011
4,63.7,27.5,2.0,2591080,Other,5/5/2011


In [88]:
df = df.merge(truthDf[['DummyID','Medview_Race','BMI']], on='DummyID')
df = df[df.Medview_Race.isin(['African American','White'])]

df = df[pd.isna(df.BMI)==False]

In [89]:
df.DummyID.value_counts().value_counts()

4    14104
8      641
2       27
3        9
1        2
6        1
Name: DummyID, dtype: int64

In [90]:
dummyCount = df.DummyID.value_counts().reset_index()
dummyCount.columns = ['DummyID','counts']

#Selecting all women with 4 images
finalDf = df[df.DummyID.isin(dummyCount[dummyCount.counts==4]['DummyID'])]


#Select the white women with duplicate images
tempDf = df[df.DummyID.isin(dummyCount[dummyCount.counts==8]['DummyID'])]
tempDf['drop'] = tempDf.filename.apply(lambda x : 'FOR' in x)  #Drop with filenames with FOR PROCESSING in it
whiteDf=tempDf[tempDf['drop']==False]
whiteDf.drop('drop',inplace = True, axis = 1)
finalDf = pd.concat([finalDf,whiteDf], axis =0)

/Users/roshansk/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
def generateDataList(dataFolder, truthFile, includeWhiteDuplicates = True):
    
    fileList = os.listdir(dataFolder)
    df = pd.DataFrame(fileList)
    df.columns = ['filename']
    df['DummyID'] = df.filename.apply(lambda x : int(x.split("_")[0]))

    truthDf = pd.read_csv(truthFile)

    df = df.merge(truthDf[['DummyID','Medview_Race','BMI']], on='DummyID')
    df = df[df.Medview_Race.isin(['African American','White'])]

    df = df[pd.isna(df.BMI)==False]


    dummyCount = df.DummyID.value_counts().reset_index()
    dummyCount.columns = ['DummyID','counts']

    #Selecting all women with 4 images
    finalDf = df[df.DummyID.isin(dummyCount[dummyCount.counts==4]['DummyID'])]

    #Select the white women with duplicate images
    if includeWhiteDuplicates ==True:
        tempDf = df[df.DummyID.isin(dummyCount[dummyCount.counts==8]['DummyID'])]
        tempDf['drop'] = tempDf.filename.apply(lambda x : 'FOR' in x)  #Drop with filenames with FOR PROCESSING in it
        whiteDf=tempDf[tempDf['drop']==False]
        whiteDf.drop('drop',inplace = True, axis = 1)
        finalDf = pd.concat([finalDf,whiteDf], axis =0)
        
        
    return finalDf

In [76]:
len(finalDf)

62108

In [115]:
finalDf.head()

,filename,DummyID,Medview_Race,BMI
0,76254864_L_MLO_1.jpg,76254864,White,23.1
1,76254864_R_CC_1.jpg,76254864,White,23.1
2,76254864_L_CC_1.jpg,76254864,White,23.1
3,76254864_R_MLO_1.jpg,76254864,White,23.1
4,76869974_L_MLO_1.jpg,76869974,African American,38.1


In [91]:
np.sum(pd.isna(finalDf.BMI))

0

In [19]:
df[df.DummyID==76075198]

,filename,DummyID,Medview_Race
37141,76075198_R_CC_1.jpg,76075198,White
37142,76075198_FOR-PROCESSING_R_MLO_1.jpg,76075198,White
37143,76075198_L_MLO_1.jpg,76075198,White
37144,76075198_FOR-PROCESSING_L_MLO_1.jpg,76075198,White
37145,76075198_R_MLO_1.jpg,76075198,White
37146,76075198_FOR-PROCESSING_L_CC_1.jpg,76075198,White
37147,76075198_L_CC_1.jpg,76075198,White
37148,76075198_FOR-PROCESSING_R_CC_1.jpg,76075198,White


In [20]:
df[df.DummyID==5201374]

,filename,DummyID,Medview_Race
34346,5201374_FOR-PROCESSING_R_MLO_1.jpg,5201374,White
34347,5201374_FOR-PROCESSING_L_CC_1.jpg,5201374,White
34348,5201374_L_CC_1.jpg,5201374,White
34349,5201374_FOR-PROCESSING_R_CC_1.jpg,5201374,White
34350,5201374_R_MLO_1.jpg,5201374,White
34351,5201374_L_MLO_1.jpg,5201374,White
34352,5201374_FOR-PROCESSING_L_MLO_1.jpg,5201374,White
34353,5201374_R_CC_1.jpg,5201374,White


In [30]:
edf = df[df.DummyID.isin(e[e.DummyID==8]['index'])]

In [31]:
len(edf)

5528

In [94]:
def getBMIData(aa, white, bmi_min, bmi_max):
    
    aaSub = aa[(aa.BMI>=bmi_min) & (aa.BMI<bmi_max)]
    whiteSub = white[(white.BMI>=bmi_min) & (white.BMI<bmi_max)]

    maxSamples = min(len(whiteSub), len(aaSub))
    
    if maxSamples==0:
        return pd.DataFrame()
    
    
    #Subsetting to keep only maxSamples number of samples in each subset
    aaSub = aaSub.sample(n=maxSamples, replace = False)
    whiteSub = whiteSub.sample(n=maxSamples, replace = False)
    
    #Figuring out the number of samples in train/valid
    numTrain = int(0.8*maxSamples)
    numValid = maxSamples - numTrain
    
    
    
    aaTrain = np.random.choice(aaSub.DummyID, numTrain,replace = False)
    whiteTrain = np.random.choice(whiteSub.DummyID, numTrain,replace = False)
    
    aaValid = np.array(list(set(aaSub.DummyID.values).difference(set(aaTrain))))
    whiteValid = np.array(list(set(whiteSub.DummyID.values).difference(set(whiteTrain))))
    
    d =  pd.concat([
        pd.DataFrame({'DummyID':aaTrain,'train':False}),
        pd.DataFrame({'DummyID':whiteTrain,'train':False}),
        pd.DataFrame({'DummyID':aaValid,'train':True}),
        pd.DataFrame({'DummyID':whiteValid,'train':True})
    ])
    
    print("Max Samples : {} numTrain  : {} df len : {}".format(maxSamples, numTrain, len(d)))
    
    return d

In [109]:
def createBMIDataset(df, numPatients, bmi_buckets = [0,20,30,40,50,55,60,100]):

    #Restricting patient base
    
    patientList = df.DummyID.unique()
    df = df[df.DummyID.isin(patientList[:numPatients])]

    
    #Creates equal number of patients from White and AA groups
    white = df[df.Medview_Race=='White']
    AA = df[df.Medview_Race=='African American']


    outputDf = pd.DataFrame()

    for i in range(len(bmi_buckets)-1):
        out = getBMIData(AA,white, bmi_buckets[i], bmi_buckets[i+1])

        outputDf = pd.concat([outputDf, out])
        
    df = df.merge(outputDf, on='DummyID')

    #Shuffling data
    index = list(range(len(df)))
    np.random.shuffle(index)
    df = df.iloc[index]
    
    return df


In [110]:
out = createBMIDataset(finalDf, 10000)

Max Samples : 540 numTrain  : 432 df len : 876
Max Samples : 8456 numTrain  : 6764 df len : 13581
Max Samples : 2284 numTrain  : 1827 df len : 3920
Max Samples : 412 numTrain  : 329 df len : 709
Max Samples : 40 numTrain  : 32 df len : 71
Max Samples : 8 numTrain  : 6 df len : 13
Max Samples : 4 numTrain  : 3 df len : 7


In [111]:
out.head()

,filename,DummyID,Medview_Race,BMI,train
31441,76676224_FOR-PROCESSING_R_CC_1.jpg,76676224,White,22.7,False
37237,3640282_L_CC_1.jpg,3640282,African American,23.3,False
19090,75767705_FOR-PROCESSING_L_MLO_1.jpg,75767705,White,28.9,False
66306,75485172_FOR-PROCESSING_R_MLO_1.jpg,75485172,White,26.8,False
52648,75775002_FOR-PROCESSING_L_MLO_1.jpg,75775002,White,28.7,False


In [112]:
out.DummyID.nunique()

7991

In [99]:
df = finalDf.copy()
bmi_buckets = [0,20,30,40,50,55,60,100]


patientList = df.DummyID.unique()

white = df[df.Medview_Race=='White']
AA = df[df.Medview_Race=='African American']


outputDf = pd.DataFrame()

for i in range(len(bmi_buckets)-1):
    out = getBMIData(AA,white, bmi_buckets[i], bmi_buckets[i+1])

    outputDf = pd.concat([outputDf, out])

Max Samples : 740 numTrain  : 592 df len : 1223
Max Samples : 13660 numTrain  : 10928 df len : 21891
Max Samples : 3720 numTrain  : 2976 df len : 6352
Max Samples : 656 numTrain  : 524 df len : 1135
Max Samples : 60 numTrain  : 48 df len : 104
Max Samples : 12 numTrain  : 9 df len : 21
Max Samples : 4 numTrain  : 3 df len : 7


In [100]:
outputDf.head()

,DummyID,train
0,4704002.0,False
1,3451937.0,False
2,75245116.0,False
3,5162527.0,False
4,75192210.0,False


In [102]:
df = df.merge(outputDf, on='DummyID')

In [103]:
len(df)

122932

In [104]:
len(outputDf)

30733

In [106]:
df.head()

,filename,DummyID,Medview_Race,BMI,train
0,76254864_L_MLO_1.jpg,76254864,White,23.1,False
1,76254864_L_MLO_1.jpg,76254864,White,23.1,False
2,76254864_L_MLO_1.jpg,76254864,White,23.1,False
3,76254864_R_CC_1.jpg,76254864,White,23.1,False
4,76254864_R_CC_1.jpg,76254864,White,23.1,False


In [107]:
patientList = df.DummyID.unique()

In [108]:
len(patientList)

12205

In [42]:
df = df[df.DummyID.isin(patientList[:])]

691.0